<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
tf.__version__
import tensorflow.keras as keras
import numpy as np

In [0]:
class Peterator(keras.utils.Sequence):

    def __init__(self, data, target_col, lookback, batch_size = 256):
        self.x, self.y = data, data[:,target_col]
        self.lookback = lookback
        self.batch_size = batch_size
        self.indices = np.arange(self.x.shape[0])

    def __len__(self):
        return math.ceil(self.x.shape[0] / self.batch_size)

    def __getitem__(self, idx):
        rows = self.indices[idx * self.batch_size + self.lookback:(idx + 1) * self.batch_size]
        samples = np.zeros((len(rows),
                            self.lookback,
                            np.shape(self.x)[-1]))
        for i, row in enumerate(rows):
            j = range(rows[i] - self.lookback, rows[i])
            samples[i] = self.x[j]
        batch_x = samples
        batch_y = self.y[rows]
        return batch_x, batch_y

In [0]:
col1, col2 = np.reshape(np.array(np.arange(0,100)), (-1, 1)), np.reshape(np.array(np.arange(100,200)), (-1, 1))
data = np.hstack((col1, col2))

In [0]:
test = Peterator(data = data, target_col = 0, lookback = 10, batch_size = 2)

In [0]:
x, y = test[0]

In [71]:
x

array([], shape=(0, 10, 2), dtype=float64)

In [60]:
y[0]

110

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

range(45, 50)

In [0]:
# Generates sequential 3D batches to feed to the model
def generator(data, lookback, delay, min_index = 0, max_index = None, 
              shuffle = False, batch_size = 128, step = 1, target_col = 0):
    # If max index not given, subtract prediction horizon - 1 (len to index) from last data point
    if max_index is None:
        max_index = len(data) - delay - 1
    # Set i to first idx with valid lookback length behind it
    i = min_index + lookback
    while 1:
        # Use shuffle for non-sequential data
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size = batch_size)
        # Else for sequential (time series)
        else:
            # Check if adding batch exceeds index bounds
            if i + batch_size >= max_index:
                # Return i to beginning
                i = min_index + lookback
            # Select next valid row range
            rows = np.arange(i, min(i + batch_size, max_index))
            # Increment i
            i += len(rows)
        # Initialize sample and target arrays
        samples = np.zeros((len(rows),
                            lookback // step,
                            np.shape(data)[-1]))
        targets = np.zeros((len(rows),))
        # Generate samples, targets
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][target_col]
        yield samples, targets